# 4. トレーニングインフラ


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

トレーニングインフラの部分について、ソクラテス式チャットボットの学習を例に説明します。

### 4.1 評価メトリクス定義
この部分は、モデルがどれだけ上手く学習できているかを測る指標を定義しています。

例えば、チャットボットが「なぜそう考えるのですか？」と質問するとき、その返答がどれだけ自然で適切かを数値化します。主に以下の2つの指標を使用：

1. loss（損失）：モデルの予測と実際の正解との差
2. perplexity（パープレキシティ）：モデルの予測の確実性を示す値。低いほど良い

### 4.2 システムリソース監視
学習中のコンピュータのリソース使用状況を監視します。これは以下のような状況を防ぐために重要です：
- メモリ不足でプログラムが突然停止する
- GPUのメモリが溢れて学習が失敗する

### 4.3 データセット分割とトレーニング設定
データを「学習用」と「評価用」に分けて、学習の設定を行います：

- 学習データ（80%）：実際にモデルが学習に使うデータ
  例：「どうしてそう思いましたか？」「その根拠は何ですか？」などの質問と回答のペア
- 評価データ（20%）：モデルの性能をテストするためのデータ

また、学習の詳細な設定も行います：
- 学習回数：30エポック
- 学習率：0.0002（モデルの学習速度）
- チェックポイントの保存：100ステップごと

### 4.4 トレーニング監視システム
学習の進行状況を監視するシステムです。以下のような機能があります：

1. 学習の安定性チェック：
   - モデルが「なぜですか？」と適切なタイミングで質問できているか
   - 質問が不自然に繰り返されていないか

2. チェックポイント保存：
   - 良い性能を示したモデルを自動的に保存
   - 問題が起きても途中から再開できるように

3. 問題検知：
   - モデルが同じ質問を繰り返すような異常を検知
   - 学習が進まない状況を検知

### 4.5 トレーナー実装
実際の学習を実行する部分です：
- メモリ管理：定期的にメモリを解放して長時間の学習を可能に
- 評価の効率化：評価時のデータ数を制限して処理を高速化
- 進捗モニタリング：学習の状況をリアルタイムで確認

これらの機能により、ソクラテス式チャットボットが適切なタイミングで適切な質問ができるように、安定した学習を実現します。

        
</div>
    </details>
</div>


In [ ]:
### 4.1 評価メトリクス定義
def compute_metrics(eval_preds):
    """基本的な評価メトリクスの計算"""
    logits, labels = eval_preds
    
    with torch.no_grad():
        # Convert logits to CPU tensor
        logits = torch.tensor(logits).cpu()
        labels = torch.tensor(labels).cpu()
        
        # Calculate perplexity
        loss = torch.nn.functional.cross_entropy(
            logits.view(-1, logits.size(-1)), 
            labels.view(-1),
            ignore_index=-100
        )
        perplexity = torch.exp(loss)
        
        # Clean up memory
        del logits, labels
        torch.cuda.empty_cache()
        
        return {
            'perplexity': perplexity.item(),
            'loss': loss.item()
        }



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

この評価メトリクス定義について、ソクラテス式チャットボットの文脈で説明させていただきます。

### 基本的な説明

この`compute_metrics`関数は、モデルの性能を評価するための指標を計算する関数です。主に2つの重要な指標を計算しています：

1. **Loss（損失）**: モデルの予測がどれだけ間違っているかを示す値
2. **Perplexity（パープレキシティ）**: モデルの予測の確実性を示す値

### 具体例で説明

例えば、ソクラテス式チャットボットの会話でこう考えてみましょう：

```
ユーザー: なぜ空は青いのですか？
ボット: その質問は興味深いですね。あなたはなぜ空が青いと思いますか？
```

このとき、モデルは：

1. **Logits（予測値）**: モデルが次に来る単語として考えられる全ての可能性とその確率
   - 例：「興味深い」(90%), 「面白い」(5%), 「難しい」(3%), その他(2%)

2. **Labels（正解）**: 実際にあるべき応答
   - 例：「興味深い」という単語が正解

### 計算の流れ

1. `with torch.no_grad():`: 
   - 評価時は学習を行わないので、メモリを節約するために勾配計算をオフにします

2. `logits = torch.tensor(logits).cpu()`:
   - 予測値をCPUメモリに移動して処理します

3. `loss = torch.nn.functional.cross_entropy()`:
   - モデルの予測と正解の差を計算します
   - 例：「興味深い」を90%の確率で予測できていれば低いloss、20%の確率だと高いloss

4. `perplexity = torch.exp(loss)`:
   - lossから計算される、モデルの「困惑度」
   - 低いほど良い（モデルが自信を持って回答している）
   - 例：
     - perplexity = 1.1 → モデルが非常に自信を持って「その質問は興味深いですね」と応答
     - perplexity = 5.0 → モデルが迷いながら応答している

### 実用的な意味

- **Loss値が低い**：モデルがソクラテス式の対話パターンをよく学習できている
- **Perplexityが低い**：モデルが自信を持って質問を返せている

例えば：
```
ユーザー: 幸せとは何だと思いますか？
ボット: （低perplexity = 自信あり）
       まず、あなたにとって幸せとは何を意味するのでしょうか？

ボット: （高perplexity = 自信なし）
       えーと...その...幸せについて...どう思いますか...？
```

このように、これらのメトリクスを監視することで、モデルがソクラテス式の対話スタイルをどれだけ上手く学習できているかを評価できます。

        
</div>
    </details>
</div>


In [ ]:

### 4.2 システムリソース監視
def log_memory_usage():
    import psutil
    import torch
    
    # CPU memory
    process = psutil.Process()
    cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
    
    # GPU memory
    gpu_memory = []
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            gpu_memory.append({
                'device': i,
                'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024,  # MB
                'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024,    # MB
                'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024  # MB
            })
    
    logging.info(f"CPU Memory usage: {cpu_memory:.2f} MB")
    for gpu in gpu_memory:
        logging.info(f"GPU {gpu['device']} Memory:")
        logging.info(f"  - Allocated: {gpu['allocated']:.2f} MB")
        logging.info(f"  - Reserved: {gpu['reserved']:.2f} MB")
        logging.info(f"  - Max Allocated: {gpu['max_allocated']:.2f} MB")

# Log dataset size
logging.info(f"Total dataset size: {len(dataset)}")
log_memory_usage()

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このシステムリソース監視の部分について、分かりやすく説明させていただきます。

### 基本的な説明
この部分のコードは、コンピュータのメモリ（記憶領域）の使用状況を監視するためのものです。大規模な言語モデルの学習では、メモリ管理が非常に重要になります。

### 主な機能

1. **CPU メモリの監視**
```python
process = psutil.Process()
cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
```
- コンピュータの主記憶域（RAM）の使用量を監視します
- MByte単位で表示されます
- 例：`CPU Memory usage: 8542.34 MB`（約8.5GB使用中）

2. **GPU メモリの監視**
```python
gpu_memory.append({
    'device': i,
    'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024,  # MB
    'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024,    # MB
    'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024  # MB
})
```
3つの重要な指標を監視します：
- `allocated`: 現在使用中のメモリ
- `reserved`: 確保されている（予約済み）メモリ
- `max_allocated`: これまでの最大使用量

### 実際の使用例
ソクラテス式チャットボットの学習時には、以下のような出力が見られるかもしれません：

```
CPU Memory usage: 12456.78 MB
GPU 0 Memory:
  - Allocated: 5234.45 MB
  - Reserved: 6000.00 MB
  - Max Allocated: 5500.67 MB
```

これは以下を意味します：
- CPUで約12.5GBのメモリを使用中
- GPUで：
  - 現在約5.2GBを使用中
  - 6GBを予約済み
  - これまでの最大使用量は約5.5GB

### メモリクリア機能
```python
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
```
この関数は、不要になったメモリを解放します：
- 例：チャットボットが長い会話を処理した後、そのデータが不要になった時
- `gc.collect()`: Python側のメモリを解放
- `torch.cuda.empty_cache()`: GPU側のメモリを解放

### なぜ重要か？
ソクラテス式チャットボットの学習では：
1. 大量の対話データを処理
2. 複雑な言語モデルを使用
3. 長い会話履歴を保持

これらが**メモリを大量に消費**するため、メモリ監視は：
- システムのクラッシュを防ぐ
- 効率的な学習を可能にする
- リソースの最適な使用を確保

という重要な役割を果たします。

        
</div>
    </details>
</div>


In [ ]:


### 4.3 データセット分割とトレーニング設定
# Split dataset into training and evaluation sets
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)
train_dataset = tokenized_dataset.select(indices[:split_idx])
# Limit evaluation dataset size
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+50])  # Maximum 50 samples

logging.info(f"Training dataset size: {len(train_dataset)}")
logging.info(f"Evaluation dataset size: {len(eval_dataset)}")

# Disable wandb via environment variable
os.environ["WANDB_DISABLED"] = "true"

# Update training arguments
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,  
    num_train_epochs=30,
    learning_rate=2e-4,           # 8e-5から2e-4に増加
    weight_decay=0.01,            # 0.06から0.01に減少
    warmup_ratio=0.1,             # 0.25から0.1に減少
    lr_scheduler_type="cosine",   # cosine_with_restartsからcosineに変更
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    gradient_accumulation_steps=8,    # バッチサイズを小さくした分、これを8に増やして補完
    max_steps=-1,
    disable_tqdm=False,
    logging_dir=LOG_OUTPUT_DIR,   
    logging_strategy="steps",
    logging_steps=10,
    no_cuda=False,
    dataloader_num_workers=1,         # CPUメモリ節約のため1に戻す
    report_to=[],
    run_name=None,
    per_device_train_batch_size=2,    # CPUメモリ節約のため2に戻す
    per_device_eval_batch_size=2,     # 同上
    gradient_checkpointing=True,
    max_grad_norm=1.0,             # 0.5から1.0に増加
    dataloader_pin_memory=True,
    save_total_limit=10,  # 10に増やす
    fp16=False,
    bf16=True,
    optim="adamw_torch",
    eval_accumulation_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model="perplexity",
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このデータセット分割とトレーニング設定について、分かりやすく説明させていただきます。

### 1. データセット分割
```python
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)
train_dataset = tokenized_dataset.select(indices[:split_idx])
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+50])
```

これは、データを「学習用」と「評価用」に分ける部分です。

例えば、1000個の対話データがあった場合：
- 学習用（train_dataset）: 800個（80%）
  - モデルが実際に学習に使うデータ
  - 例：「なぜ勉強は大切ですか？」→「あなたはなぜ勉強が大切だと考えていますか？」

- 評価用（eval_dataset）: 50個
  - モデルの性能をテストするためのデータ
  - 学習には使わない
  - 最大50個に制限（メモリ効率のため）

### 2. トレーニング設定（TrainingArguments）

重要なパラメータを見ていきましょう：

#### 基本設定
```python
num_train_epochs=30,              # 30回全データを学習
learning_rate=2e-4,              # 学習率（どれだけ大きく更新するか）
per_device_train_batch_size=2,    # 一度に処理する対話数
```

#### 学習の進め方
```python
evaluation_strategy="steps",      # 100ステップごとに評価
eval_steps=100,
save_strategy="steps",           # 100ステップごとに保存
save_steps=100,
```

例えば：
1. 100個の対話を処理
2. モデルの性能を評価
3. モデルを保存
4. また100個処理...という流れ

#### メモリ管理関連
```python
gradient_accumulation_steps=8,    # 8回分の計算をまとめて更新
per_device_train_batch_size=2,    # 一度に2つの対話を処理
dataloader_num_workers=1,         # データ読み込みの並列処理数
```

これは、限られたメモリで効率よく学習するための設定です。

#### 学習の最適化設定
```python
weight_decay=0.01,               # モデルの複雑さを抑制
warmup_ratio=0.1,               # 学習率を徐々に上げる期間
lr_scheduler_type="cosine",      # 学習率の変化パターン
```

ソクラテス式チャットボットの文脈で例えると：
1. 最初は慎重に学習（warmup）
   - 「簡単な質問への返し方」から始める
2. 徐々に本格的な学習
   - 「より深い問いかけ」の練習
3. 最後は微調整
   - 「洗練された対話」の完成

#### モデルの保存設定
```python
load_best_model_at_end=True,     # 最も性能の良かったモデルを保存
metric_for_best_model="perplexity", # perplexityが最も良いモデルを選択
```

例：
- perplexity = 1.2 のモデル → 自信を持って対話できる
- perplexity = 4.0 のモデル → 迷いがある対話

より良い方（低いperplexity）のモデルを最終的に採用します。

これらの設定により、効率的かつ効果的にソクラテス式の対話スタイルを学習できるようになっています。

        
</div>
    </details>
</div>


In [ ]:

### 4.4 トレーニング監視システム実装
class TrainingMonitorCallback(TrainerCallback):
    def __init__(self):
        self.train_start_time = None
        self.metrics_history = {
            'step': [],
            'train_loss': [],
            'eval_loss': [],
            'learning_rate': [],
            'perplexity': [],
            'grad_norm': [],
            'gpu_memory_usage': [],
        }
        self.output_dir = Path(f"{BASE_OUTPUT_DIR}/training_progress")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # 安定性監視用の設定
        self.window_size = 5  # 移動平均のウィンドウサイズ
        self.last_checkpoint_step = 0
        self.min_steps_between_checkpoints = 100  # チェックポイント間の最小ステップ数
        self.stable_checkpoints = []  # 安定したチェックポイントを記録
        
        # メトリクスの閾値設定
        self.perplexity_threshold = 2.5  # 良好なperplexityの閾値
        self.eval_loss_variance_threshold = 0.1  # eval_lossの許容変動幅
        self.grad_norm_bounds = (0.1, 2.0)  # grad_normの適正範囲
        self.max_stable_checkpoints = 5  # 安定チェックポイントの最大数を設定
        self.error_count = 0  # エラー回数を追跡
        self.max_consecutive_errors = 3  # 連続エラーの許容回数
        
        # バランス監視用の設定を追加
        self.variance_bias_window = 10  # より長いウィンドウで傾向を見る
        self.train_losses = []  # 訓練損失の履歴
        self.eval_losses = []   # 評価損失の履歴
        self.optimal_gap_range = (0.1, 0.3)  # 訓練損失と評価損失の理想的な差分範囲
    
    def _calculate_stability_metrics(self, state):
        """安定性メトリクスを計算"""
        if len(self.metrics_history['perplexity']) < self.window_size:
            return None
            
        recent_perplexity = self.metrics_history['perplexity'][-self.window_size:]
        recent_eval_loss = self.metrics_history['eval_loss'][-self.window_size:]
        recent_grad_norm = self.metrics_history['grad_norm'][-self.window_size:]
        
        # 移動平均と標準偏差を計算
        perplexity_mean = np.mean(recent_perplexity)
        eval_loss_std = np.std(recent_eval_loss)
        grad_norm_mean = np.mean(recent_grad_norm)
        
        return {
            'perplexity_mean': perplexity_mean,
            'eval_loss_std': eval_loss_std,
            'grad_norm_mean': grad_norm_mean
        }
    
    def _calculate_variance_bias_metrics(self):
        """分散と偏りのバランスを計算"""
        if len(self.train_losses) < self.variance_bias_window or \
           len(self.eval_losses) < self.variance_bias_window:
            return None
            
        recent_train = self.train_losses[-self.variance_bias_window:]
        recent_eval = self.eval_losses[-self.variance_bias_window:]
        
        # 訓練損失と評価損失の差（バイアスの指標）
        loss_gap = np.mean(recent_eval) - np.mean(recent_train)
        
        # 損失の変動（分散の指標）
        train_variance = np.var(recent_train)
        eval_variance = np.var(recent_eval)
        
        return {
            'loss_gap': loss_gap,
            'train_variance': train_variance,
            'eval_variance': eval_variance,
            'total_variance': (train_variance + eval_variance) / 2
        }
    
    def _is_balanced_state(self, variance_bias_metrics):
        """バランスの取れた状態かを判断"""
        if variance_bias_metrics is None:
            return False
            
        # 理想的な差分範囲内にあるか
        good_gap = (self.optimal_gap_range[0] <= variance_bias_metrics['loss_gap'] <= self.optimal_gap_range[1])
        
        # 分散が適度に小さいか
        stable_variance = variance_bias_metrics['total_variance'] < 0.1
        
        # 訓練と評価の分散が近いか（安定性の指標）
        variance_ratio = min(variance_bias_metrics['train_variance'], variance_bias_metrics['eval_variance']) / \
                        max(variance_bias_metrics['train_variance'], variance_bias_metrics['eval_variance'])
        balanced_variance = variance_ratio > 0.7  # 70%以上の類似性
        
        return good_gap and stable_variance and balanced_variance
    
    def _should_save_checkpoint(self, state, metrics):
        """チェックポイント保存の判断を拡張"""
        # 既存の条件をチェック
        basic_conditions = super()._should_save_checkpoint(state, metrics)
        
        # バランス状態もチェック
        variance_bias_metrics = self._calculate_variance_bias_metrics()
        balanced_state = self._is_balanced_state(variance_bias_metrics)
        
        if balanced_state:
            logging.info(f"Found balanced state at step {state.global_step}")
            logging.info(f"Variance-Bias metrics: {variance_bias_metrics}")
        
        return basic_conditions or balanced_state  # どちらかの条件を満たせば保存
    
    def _safe_save_checkpoint(self, checkpoint_dir, state, metrics, stability_metrics):
        """安全にチェックポイントを保存"""
        try:
            # チェックポイントディレクトリの作成を試みる
            os.makedirs(checkpoint_dir, exist_ok=True)
            
            # モデルの保存を試みる
            try:
                self.trainer.save_model(checkpoint_dir)
            except Exception as e:
                logging.error(f"Failed to save model checkpoint: {str(e)}")
                return False
            
            # メトリクス情報の保存を試みる
            try:
                metrics_path = os.path.join(checkpoint_dir, "stability_metrics.json")
                with open(metrics_path, 'w') as f:
                    json.dump({
                        'step': state.global_step,
                        'metrics': stability_metrics,
                        'eval_metrics': metrics,
                        'timestamp': datetime.now().isoformat()
                    }, f, indent=2)
            except Exception as e:
                logging.error(f"Failed to save metrics: {str(e)}")
                # メトリクスの保存に失敗してもチェックポイントは有効
                
            return True
            
        except Exception as e:
            self.error_count += 1
            logging.error(f"Checkpoint creation failed (attempt {self.error_count}): {str(e)}")
            if self.error_count >= self.max_consecutive_errors:
                logging.warning("Too many consecutive checkpoint errors. Will skip future checkpoint attempts.")
            return False
    
    def _safe_remove_checkpoint(self, checkpoint_path):
        """安全にチェックポイントを削除"""
        try:
            if os.path.exists(checkpoint_path):
                shutil.rmtree(checkpoint_path)
                logging.info(f"Successfully removed old checkpoint: {checkpoint_path}")
        except Exception as e:
            logging.error(f"Failed to remove old checkpoint {checkpoint_path}: {str(e)}")
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """評価時のチェックポイント判断"""
        if not metrics:
            return
        
        try:
            stability_metrics = self._calculate_stability_metrics(state)
            if stability_metrics and self._should_save_checkpoint(state, stability_metrics):
                # 安定したチェックポイントとして保存を試みる
                checkpoint_dir = os.path.join(
                    args.output_dir,
                    f"stable_checkpoint-{state.global_step}"
                )
                
                if self._safe_save_checkpoint(checkpoint_dir, state, metrics, stability_metrics):
                    self.stable_checkpoints.append({
                        'step': state.global_step,
                        'path': checkpoint_dir,
                        'metrics': stability_metrics
                    })
                    self.last_checkpoint_step = state.global_step
                    self.error_count = 0  # 成功したらエラーカウントをリセット
                    logging.info(f"Saved stable checkpoint at step {state.global_step}")
                    logging.info(f"Stability metrics: {stability_metrics}")
                
        except Exception as e:
            logging.error(f"Error during evaluation callback: {str(e)}")
            # エラーが発生しても処理を継続
    
    def on_train_end(self, args, state, control, **kwargs):
        """トレーニング終了時の処理"""
        try:
            training_duration = datetime.now() - self.train_start_time
            
            # 安定したチェックポイントの概要を保存
            try:
                checkpoints_summary = os.path.join(self.output_dir, 'stable_checkpoints_summary.json')
                with open(checkpoints_summary, 'w') as f:
                    json.dump({
                        'total_checkpoints': len(self.stable_checkpoints),
                        'checkpoints': self.stable_checkpoints
                    }, f, indent=2)
            except Exception as e:
                logging.error(f"Failed to save checkpoints summary: {str(e)}")
            
            # メトリクス履歴の保存を試みる
            try:
                history_file = self.output_dir / 'training_metrics.json'
                with open(history_file, 'w', encoding='utf-8') as f:
                    json.dump(self.metrics_history, f, indent=2)
            except Exception as e:
                logging.error(f"Failed to save training metrics: {str(e)}")
            
            # 終了ログの出力
            logging.info(f"Training completed. Duration: {training_duration}")
            self._log_final_metrics()
            
        except Exception as e:
            logging.error(f"Error during training end callback: {str(e)}")
    
    def _log_final_metrics(self):
        """最終メトリクスのログ出力"""
        try:
            if self.metrics_history['train_loss']:
                logging.info(f"Final training loss: {self.metrics_history['train_loss'][-1]:.4f}")
            if self.metrics_history['eval_loss']:
                logging.info(f"Final evaluation loss: {self.metrics_history['eval_loss'][-1]:.4f}")
            if self.metrics_history['perplexity']:
                logging.info(f"Final perplexity: {self.metrics_history['perplexity'][-1]:.4f}")
            logging.info(f"Total stable checkpoints saved: {len(self.stable_checkpoints)}")
        except Exception as e:
            logging.error(f"Error logging final metrics: {str(e)}")



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



トレーニング監視システムについて、ソクラテス式チャットボットの文脈で分かりやすく説明させていただきます。

### 1. 基本的な監視項目（初期化部分）
```python
self.metrics_history = {
    'step': [],            # 学習のステップ数
    'train_loss': [],      # 学習時の誤差
    'eval_loss': [],       # 評価時の誤差
    'learning_rate': [],   # 学習率
    'perplexity': [],      # モデルの確信度
    'grad_norm': [],       # 学習の安定性
    'gpu_memory_usage': [] # GPUメモリ使用量
}
```

これは、学習の進行状況を記録する「学習日誌」のようなものです。

### 2. 安定性の監視
```python
# 安定性監視用の設定
self.window_size = 5  # 直近5回分の結果を見る
self.perplexity_threshold = 2.5  # perplexityの目標値
```

例えば、ソクラテス式の対話で：
- 直近5回の対話で：
  - perplexity < 2.5 → 「安定して質問を返せている」
  - perplexity > 2.5 → 「まだ迷いがある」

### 3. 学習のバランス監視
```python
self.optimal_gap_range = (0.1, 0.3)  # 訓練と評価の理想的な差
```

これは「過学習」を防ぐための監視です。

例：
```
訓練データでの会話：
ユーザー: なぜ運動は大切ですか？
ボット: あなたにとって運動とはどのような意味を持ちますか？
（Loss: 0.2）

評価データでの会話：
ユーザー: なぜ読書は重要ですか？
ボット: 読書の重要性について、あなたはどのようにお考えですか？
（Loss: 0.4）

差分: 0.2 → 適切なバランス
```

### 4. 安定性の計算（_calculate_stability_metrics）
```python
def _calculate_stability_metrics(self, state):
    recent_perplexity = self.metrics_history['perplexity'][-self.window_size:]
    # ...
```

直近の結果を分析して、モデルの安定性を確認します：
- perplexityの平均値
- 評価時の誤差のばらつき
- 学習の安定度

### 5. チェックポイントの保存判断
```python
def _should_save_checkpoint(self, state, metrics):
    variance_bias_metrics = self._calculate_variance_bias_metrics()
    balanced_state = self._is_balanced_state(variance_bias_metrics)
```

以下の場合にモデルを保存します：
1. 基本条件を満たす（一定間隔）
2. 特に良い結果が出た時

例：
```
Step 1000:
- perplexity: 2.0（良好）
- 学習と評価のバランスが取れている
→ このモデルを保存
```

### 6. トレーニング終了時の処理
```python
def on_train_end(self, args, state, control, **kwargs):
    # 学習結果のまとめを保存
```

最終的な学習結果をまとめます：
- 総学習時間
- 最終的な性能
- 保存したチェックポイントの一覧

例：
```
学習完了レポート：
- 学習時間: 12時間30分
- 最終perplexity: 1.8
- 安定チェックポイント: 5個保存
```

このように、モデルの学習過程を細かく監視し、問題があれば早期に発見できるようになっています。特にソクラテス式の対話モデルでは、一貫性のある質問の仕方を学習できているかが重要なので、この監視システムが重要な役割を果たします。

        
</div>
    </details>
</div>


In [ ]:

### 4.5 トレーナー実装と初期化
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)

class CustomTrainer(Trainer):
    def training_step(self, *args, **kwargs):
        loss = super().training_step(*args, **kwargs)
        if self.state.global_step % 50 == 0:
            clear_memory()
            gc.collect()
            torch.cuda.empty_cache()
        return loss

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        if eval_dataset is not None:
            # Limit evaluation dataset to 100 samples
            eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
        return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)

# Trainer initialization
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TrainingMonitorCallback()],
)



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



トレーナーの実装と初期化について、ソクラテス式チャットボットの文脈で分かりやすく説明させていただきます。

### 1. データコレーター（データの整形役）
```python
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,           # マスク言語モデリングを使用しない
    pad_to_multiple_of=8 # データを8の倍数に揃える
)
```

これは、対話データを学習に適した形に整えるツールです。

例えば：
```
入力データ：
ユーザー: 幸せとは何ですか？
ボット: あなたにとって幸せとは何を意味しますか？

↓ データコレーターが処理

[トークン化されたデータ]
[1234, 5678, 9012, ...] （8の倍数長に調整）
```

### 2. カスタムトレーナー
```python
class CustomTrainer(Trainer):
    def training_step(self, *args, **kwargs):
        loss = super().training_step(*args, **kwargs)
        if self.state.global_step % 50 == 0:  # 50ステップごとに
            clear_memory()                     # メモリクリア
        return loss
```

通常のトレーナーを改良して：
1. メモリ管理を強化
2. 評価データサイズを制限

例えば：
```
学習の流れ：
Step 1-49: 通常の学習
Step 50: メモリクリア
Step 51-99: 通常の学習
Step 100: メモリクリア
...
```

### 3. 評価機能のカスタマイズ
```python
def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
    if eval_dataset is not None:
        # 評価データを最大100サンプルに制限
        eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
```

評価時の工夫：
- 最大100の対話のみを評価
- メモリ使用量を抑制
- 評価時間を短縮

例：
```
全評価データ: 500対話
↓
実際に使用: 100対話
- 「なぜ...」で始まる質問への応答
- 「どのように...」で始まる質問への応答
など、バランスよく選択
```

### 4. トレーナーの初期化
```python
trainer = CustomTrainer(
    model=model,                          # 学習するモデル
    args=training_args,                   # 学習設定
    train_dataset=train_dataset,          # 学習データ
    eval_dataset=eval_dataset,            # 評価データ
    data_collator=data_collator,         # データ整形ツール
    compute_metrics=compute_metrics,      # 評価指標の計算
    callbacks=[TrainingMonitorCallback()] # 学習監視システム
)
```

これは「先生」を設定するようなものです：

1. **教材の準備**
   - train_dataset: 練習用の対話集
   - eval_dataset: テスト用の対話集

2. **教え方の設定**
   - training_args: 学習のペース、方法
   - data_collator: 教材の整理方法

3. **進捗管理**
   - compute_metrics: テストの採点方法
   - TrainingMonitorCallback: 学習の記録係

実際の学習例：
```
Step 1: 基本的な問いかけの練習
ユーザー: 何故ですか？
ボット: もう少し具体的に教えていただけますか？

Step 100: より深い問いかけの練習
ユーザー: 幸せとは何ですか？
ボット: あなたにとって、幸せはどのような形で現れますか？

Step 1000: 複雑な対話の練習
ユーザー: AIと人間の関係性について
ボット: その問いは興味深いですね。AIと人間の関係について、
あなたはどのようなお考えをお持ちですか？
```

このように、効率的かつ効果的な学習を実現するための「先生」の役割を果たします。

        
</div>
    </details>
</div>
